# EC数据的查询



In [25]:
%%time
import pymssql 
import pandas as pd

## 连接数据库
server = '172.21.158.201'    # 连接服务器地址
user = 'down'# 连接帐号
password = 'downx'# 连接密码
conn = pymssql.connect(server, user, password, 'ZJSZDZDB')  #获取连接

sql_all = '''
select IIiii,lat,lon FROM TAB_StationInfo
WHERE (IIiii  IN  ('58566','58567','58564','58569','58556','58555','58560','58654','58657','58658','58763','58656','58761','58559','58568','58660','58662','58663','58653','58652','58655','58665','58661','58666','58664','58669','58667','58668','K8705','K8706','K8818','K8609','K8821','K8611','K8903','K8910','K8282','K8217','K8201','K8301','K8413','K8505'))
'''
df = pd.read_sql(sql_all , con=conn)

CPU times: user 6.92 ms, sys: 515 µs, total: 7.44 ms
Wall time: 22.9 ms


# 编写自动站数据的类

In [75]:
%%time
import pymysql
import pandas as pd
import os
import xarray as xr
import numpy as np

class sql_plot:
    def __init__(self, start_time,end_time,select_type):
        self.start =  start_time 
        self.end = end_time
        self.type = select_type
        self.station = ('58566','58567','58564','58569','58556','58555','58560','58654','58657','58658','58763','58656','58761','58559','58568','58660','58662','58663','58653','58652','58655','58665','58661','58666','58664','58669','58667','58668','K8705','K8706','K8818','K8609','K8821','K8611','K8903','K8910','K8282','K8217','K8201','K8301','K8413','K8505')
        self.Aws = ('58566','58567','58564','58569','58556','58555','58560','58654','58657','58658','58763','58656','58761','58559','58568','58660','58662','58663','58653','58652','58655','58665','58661','58666','58664','58669','58667','58668')
        self.Mws = ('K8705','K8706','K8818','K8609','K8821','K8611','K8903','K8910','K8282','K8217','K8201','K8301','K8413','K8505')  
        self.server = '127.0.0.1' 
        self.user = 'root'
        self.password = '051219'# 连接密码   
        self.port = 3306
        self.database = "ZJSZDZDB"
    def return_zdz_data(self):
        self.start ="'" +  self.start + "'" 
        self.end = "'" +  self.end + "'" 
        conn = pymysql.connect(
            host = self.server, 
            port = self.port,
            user = self.user, 
            password = self.password,
            database = self.database)
        sql_Aws = f'''select a.IIiii,b.lat,b.lon,SUM(a.RR)/10.0
            from TAB_Aws2019 AS a left join TAB_StationInfo AS b on a.IIiii=b.IIiii 
            WHERE (b.IIiii  IN  {self.Aws} and (tTime BETWEEN {self.start} and {self.end} )) 
            GROUP BY a.IIiii,lat,lon'''
        sql_Mws = f'''select a.IIiii,b.lat,b.lon,SUM(a.RR)/10.0
            from TAB_Mws2019 AS a left join TAB_StationInfo AS b on a.IIiii=b.IIiii 
            WHERE (b.IIiii  IN  {self.Mws} and (tTime BETWEEN {self.start} and {self.end} )) 
            GROUP BY a.IIiii,lat,lon '''
        dfa = pd.read_sql(sql_Aws, con=conn)
        dfm = pd.read_sql(sql_Mws, con=conn)
        df = pd.concat([dfa,dfm])
        # 输出数据
        data_canvas = {
            "station_list": [],
            "station": []
        }
        for i in range(df.shape[0]):
            station_data = []
            station_data.append(round(df.iloc[i, 2],2))#lon
            station_data.append(round(df.iloc[i, 1],2))#lat
            value = round(df.iloc[i, 3],2)
            if value!=-9999:
                station_data.append(value)
                data_canvas['station_list'].append(df.iloc[i,0])
                data_canvas['station'].append(station_data)
        canvas_data = data_canvas
        return canvas_data
    def return_ec_data(self):
        # 数据---文档
        test_time = '2022041700'
        file_path = "/home/workspace/Data/My_Git/" + test_time + "/"
        # 文件--数据
        conn = pymysql.connect(
            host = self.server, 
            port = self.port,
            user = self.user, 
            password = self.password,
            database = self.database)
        sql_location = f'''
            select IIiii,lat,lon from TAB_StationInfo
            where (IIiii in  {self.station} )     
        '''
        df = pd.read_sql(sql_location, con=conn)
        # 查询ec数据
        files = os.listdir(file_path)  
        fstart = xr.open_dataset(file_path + files[int(self.start)])
        tp_start = fstart.tp
        fend = xr.open_dataset(file_path + files[int(self.end)])
        tp_end = fend.tp 
        tp = tp_end.data - tp_start.data
        # 输出数据
        data_canvas = {
            "station_list": [],
            "station": []
        }
        for i in range(df.shape[0]):
            station_data = []
            
            station_data.append(round(df.iloc[i, 2],2))#lon
            station_data.append(round(df.iloc[i, 1],2))#lat
            # 求数据之差
            tp0 = tp_end.sel(lonS=round(df.iloc[i, 2],2), latS=round(df.iloc[i, 1],2),method='nearest').data.tolist()[0]
            print(type(tp0))
            tp1 = tp_start.sel(lonS=round(df.iloc[i, 2],2), latS=round(df.iloc[i, 1],2),method='nearest').data[0]
            value = round((tp0-tp1),2)
            if not np.isnan(value):
                station_data.append(value)
                data_canvas['station_list'].append(df.iloc[i,0])
                data_canvas['station'].append(station_data)
        canvas_data = data_canvas
        return canvas_data
    def return_data(self):
        if self.type == "zdz":
            canvas_data = self.return_zdz_data()
        else:
            canvas_data = self.return_ec_data()
        return canvas_data

# zdz    
#start_time,end_time,select_type = "2019-03-10 20:00","2019-03-15 20:00","zdz"
# ec
start_time,end_time,select_type = 1,25,"ec"
sql_worker = sql_plot(start_time,end_time,select_type)
data = sql_worker.return_data()

CPU times: user 139 ms, sys: 344 µs, total: 140 ms
Wall time: 161 ms


In [73]:
data

{'station_list': [58559,
  58568,
  58652,
  58653,
  58660,
  58661,
  58662,
  58663,
  58664,
  58665,
  58666,
  58667,
  58668,
  58669,
  'K8201',
  'K8217',
  'K8282',
  'K8301',
  'K8413',
  'K8505',
  'K8609',
  'K8611',
  'K8705',
  'K8706',
  'K8818',
  'K8821',
  'K8903',
  'K8910'],
 'station': [[120.97, 29.15, 21.4],
  [121.38, 29.12, 22.3],
  [120.72, 28.87, 19.1],
  [120.92, 28.81, 22.5],
  [121.19, 28.87, 24.0],
  [121.51, 28.66, 23.9],
  [121.71, 29.01, 28.7],
  [121.92, 28.72, 20.3],
  [121.36, 28.37, 31.6],
  [121.42, 28.62, 25.5],
  [121.9, 28.45, 10.0],
  [121.27, 28.08, 18.7],
  [121.52, 28.1, 21.1],
  [121.62, 28.26, 23.9],
  [121.28, 28.65, 26.8],
  [121.17, 28.63, 24.9],
  [120.98, 28.59, 24.3],
  [121.35, 28.58, 29.3],
  [121.56, 28.45, 27.8],
  [121.24, 28.13, 29.6],
  [121.45, 28.85, 28.9],
  [121.64, 28.71, 28.3],
  [120.79, 29.05, 0.6],
  [121.0, 29.07, 23.7],
  [121.31, 29.0, 27.9],
  [121.58, 28.93, 30.8],
  [120.79, 28.72, 20.0],
  [120.34, 28.7, 21.7]

In [36]:
import pymysql
import pandas as pd
import os
import xarray as xr
test_time = '2022041700'
file_path = "/home/workspace/Data/My_Git/" + test_time + "/"
files = os.listdir(file_path)  
print(files)
fstart = xr.open_dataset(file_path + files[15])

['ecfine.I2022041700.000.F2022041700.nc', 'ecfine.I2022041700.003.F2022041703.nc', 'ecfine.I2022041700.006.F2022041706.nc', 'ecfine.I2022041700.009.F2022041709.nc', 'ecfine.I2022041700.012.F2022041712.nc', 'ecfine.I2022041700.015.F2022041715.nc', 'ecfine.I2022041700.018.F2022041718.nc', 'ecfine.I2022041700.021.F2022041721.nc', 'ecfine.I2022041700.024.F2022041800.nc', 'ecfine.I2022041700.027.F2022041803.nc', 'ecfine.I2022041700.030.F2022041806.nc', 'ecfine.I2022041700.033.F2022041809.nc', 'ecfine.I2022041700.036.F2022041812.nc', 'ecfine.I2022041700.039.F2022041815.nc', 'ecfine.I2022041700.042.F2022041818.nc', 'ecfine.I2022041700.045.F2022041821.nc', 'ecfine.I2022041700.048.F2022041900.nc', 'ecfine.I2022041700.051.F2022041903.nc', 'ecfine.I2022041700.054.F2022041906.nc', 'ecfine.I2022041700.057.F2022041909.nc', 'ecfine.I2022041700.060.F2022041912.nc', 'ecfine.I2022041700.063.F2022041915.nc', 'ecfine.I2022041700.066.F2022041918.nc', 'ecfine.I2022041700.069.F2022041921.nc', 'ecfine.I202204

In [17]:
import pymssql 
import pandas as pd

## 连接数据库
server = '172.21.158.201'    # 连接服务器地址
user = 'down'# 连接帐号
password = 'downx'# 连接密码
conn = pymssql.connect(server, user, password, 'ZJSZDZDB')  #获取连接

## 以查询区域站数据为例 ---- 以下是说明（国家站对应TAB_Mws2022改为TAB_Aws2022）
# RR是小时降水，R01是第一分钟的降水,以此类推
# TAB_Mws2022是对应每年的数据表，具体2022可以更改对应年份------》“from TAB_Mws2022 as a left join TAB_StationInfo”
# TAB_Mws2022是2022年区域站的数据，若想查询国家站数据则需要改为TAB_Aws2022
# tTime是时间，查询对应时间就修改成对应的时段-------》'tTime between '2022-01-10 20:00' and '2022-02-10 08:00''
sql_M_location = 'select RR,R01,R02,R03,R04,R05,R06,R07,R08,R09,R10\
    ,R11,R12,R13,R14,R15,R16,R17,R18,R19,R20,\
    R21,R22,R23,R24,R25,R26,R27,R28,R29,R30,\
    R31,R32,R33,R34,R35,R36,R37,R38,R39,R40,\
    R41,R42,R43,R44,R45,R46,R47,R48,R49,R50,\
    R51,R52,R53,R54,R55,R56,R57,R58,R59,R60 \
    from TAB_Mws2022 as a left join TAB_StationInfo as b on a.IIiii=b.IIiii where\
    (b.IIiii in (select IIiii from TAB_StationInfo where(City = '台州') and\
    tTime between '2022-01-10 20:00' and '2022-02-10 08:00'))'

df_M_location = pd.read_sql(sql_M_location , con=conn)

In [18]:
df_M_location

,RR,R01,R02,R03,R04,R05,R06,R07,R08,R09,...,R51,R52,R53,R54,R55,R56,R57,R58,R59,R60
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338439,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
338440,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
338441,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
338442,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
%%time
# coding=UTF-8
import gevent
from math import isnan
import numpy as np
#import modin.pandas as pd 
import pandas as pd
import time
import netCDF4 
# import h5netcdf.legacyapi as netCDF4
import h5py

from mpl_toolkits.basemap import Basemap
from matplotlib.path import Path
from matplotlib.patches import PathPatch
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import shapefile
import matplotlib as mpl
import xarray as xr
from matplotlib.font_manager import FontProperties
import netCDF4 as nc
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib
import geopandas as gpd
from ncmaps import Cmaps
from io import BytesIO
import base64
import json
import math
from scipy.interpolate import griddata
from math import ceil, floor
from rasterio import features
from affine import Affine
import os
from datetime import *
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
from matplotlib.font_manager import FontProperties  # 步骤一
from rasterio import features
import pymysql
import pymssql
import numpy as np
import pandas as pd
import pickle
import redis
import datetime as dtt

class nlcmap(LinearSegmentedColormap):
    """A nonlinear colormap"""

    name = 'nlcmap'

    def __init__(self, cmap, levels):
        self.cmap = cmap
        self.monochrome = self.cmap.monochrome
        self.levels = np.asarray(levels, dtype='float64')
        self._x = self.levels/ self.levels.max()
        self.levmax = self.levels.max()
        self.levmin = self.levels.min()
        self._y = np.linspace(self.levmin, self.levmax, len(self.levels))

    def __call__(self, xi, alpha=1.0, **kw):
        yi = np.interp(xi, self._x, self._y)
        return self.cmap(yi/self.levmax, alpha)
class station_plot:
    def __init__(self):
        self.conn = pymysql.connect(host="127.0.0.1",port=3306,user="root",passwd="051219",db="ZJSZDZDB")
        self.shp_path = "shpfile/country/"
    # 外部函数
    def colormap(self,plot_value,color_label):
        '''色标的自定义'''
        plt.rcParams['axes.facecolor']='snow'
        # 降水
        if plot_value=="rain":
            if color_label =="rain_24hours":
                colorslist = ['#FFFFFF','#A6F28f','#3DBA3D',"#61B8FF","#0000E1","#FA00FA","#800040"]# 24降水
                levels = [0,1,10,25,50,100,250,1000]
                cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=7)
                cmap_nonlin = nlcmap(cmaps, levels)
            elif color_label =="rain_12hours":
                colorslist = ['#FFFFFF','#A6F28f','#3DBA3D',"#61B8FF","#0000E1","#FA00FA","#800040"]# 12降水
                levels = [0,1,5,15,30,70,140,250]
                cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=7)
                cmap_nonlin = nlcmap(cmaps, levels)
            elif color_label =="rain_06hours":
                colorslist = ['#FFFFFF','#A6F28f','#3DBA3D',"#61B8FF","#0000E1","#FA00FA","#800040"]# 06降水
                levels = [0,1,4,13,25,60,120,250]
                cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=7)
                cmap_nonlin = nlcmap(cmaps, levels)
            elif color_label =="rain_03hours":
                colorslist = ['#FFFFFF','#A6F28f','#3DBA3D',"#61B8FF","#0000E1","#FA00FA","#800040"]# 03降水
                levels = [0,1,3,10,20,50,70,150]
                cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=7)
                cmap_nonlin = nlcmap(cmaps, levels)
            elif color_label =="rain_01hours":
                colorslist = ['#FFFFFF','#A6F28f','#3DBA3D',"#61B8FF","#0000E1","#FA00FA","#800040"]# 01降水
                levels = [0,1,2,7,15,40,50,100]
                cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=7)
                cmap_nonlin = nlcmap(cmaps, levels)
        elif plot_value=="tmax" or plot_value=="tmin":
            if color_label =="temp_normal":
                #colorslist = ['#1F1FFF',"#3B3BFF","#5757FF","#7272FF","#8F8FFF","#ABABFF","#C7C7FF","#E3E3FF","#FDFC8B","#F8E08B","#F3C36F","#EFA76E","#EC8A51","#F31717"]# 气温
                #colornum = len(colorslist)
                level = list(np.linspace(self.min-1, self.max+1, num=14, endpoint=True, retstep=False, dtype=None))
                levels = [round(i,1) for i in level]
                cmap_nonlin = 'seismic'#'coolwarm'#
            elif color_label =="temp_high":
                level = list(np.linspace(self.min-1, self.max+1, num=14, endpoint=True, retstep=False, dtype=None))
                levels = [round(i,1) for i in level]
                cmap_nonlin = 'Reds'
            elif color_label =="temp_low":
                level = list(np.linspace(self.min-1, self.max+1, num=14, endpoint=True, retstep=False, dtype=None))
                levels = [round(i,1) for i in level]
                cmap_nonlin = 'Blues_r'
        elif plot_value=="wind":
            if color_label =="wind_normal":
                colorslist = ['#FFFFFF','#A6F28f','#3DBA3D',"#61B8FF","#0000E1","#FA00FA","#800040"]# 风力
                levels = [0,1,10,15,25,50,100,250]
                cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=7)
                cmap_nonlin = nlcmap(cmaps, levels) 
            elif color_label =="wind_other":
                colorslist = ['#FFFFFF','#A6F28f','#3DBA3D',"#61B8FF","#0000E1","#FA00FA","#800040"]# 风力其他
                levels = [0,1,10,15,25,50,100,250]
                cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=7)
                cmap_nonlin = nlcmap(cmaps, levels)
        elif plot_value=="view":
            if color_label =="view_normal":
                colorslist = ['#A93434','#FF9600','#FFFD37',"#55FF37","#ABF3D3","#5EC6EB","#B1F1EF"]# 能见度
                levels = [0,50,200,500,1000,1500,3000,20000]
                cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=7)
                cmap_nonlin = nlcmap(cmaps, levels) 
            elif color_label =="view_other":
                colorslist = ['#FFFFFF','#FF9600','#3DBA3D',"#61B8FF","#0000E1","#FA00FA","#800040"]# 能见度
                levels = [0,1,10,15,25,50,100,250]
                cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=7)
                cmap_nonlin = nlcmap(cmaps, levels)             
        return cmap_nonlin ,levels
    def city_shp(self,data_xr):
        shp_da = self.add_shape_coord_from_data_array(data_xr,self.shp_path+"taizhou.shp", "country")
        awash_da = shp_da.where(shp_da.country<7, other=np.nan)
        return awash_da
    def transform_from_latlon(self,lat, lon):
        lat = np.asarray(lat)
        lon = np.asarray(lon)
        trans = Affine.translation(lon[0], lat[0])
        scale = Affine.scale(lon[1] - lon[0], lat[1] - lat[0])
        return trans * scale    
    def rasterize(self,shapes, coords, latitude='lat', longitude='lon',fill=np.nan, **kwargs):
        transform = self.transform_from_latlon(coords[latitude], coords[longitude])
        out_shape = (len(coords[latitude]), len(coords[longitude]))
        raster = features.rasterize(shapes, out_shape=out_shape,
                                fill=fill, transform=transform,
                                dtype=float, **kwargs)
        spatial_coords = {latitude: coords[latitude], longitude: coords[longitude]}
        return xr.DataArray(raster, coords=spatial_coords, dims=(latitude, longitude))
    def add_shape_coord_from_data_array(self,xr_da, shp_path, coord_name):   
        shp_gpd = gpd.read_file(shp_path)
        shapes = [(shape, n) for n, shape in enumerate(shp_gpd.geometry)]
        xr_da[coord_name] = self.rasterize(shapes, xr_da.coords, longitude='lon', latitude='lat')
        return xr_da
    def basemask(self,cs, ax, map, shpfile):
        sf = shapefile.Reader(shpfile)
        vertices = []
        codes = []
        for shape_rec in sf.shapeRecords():
            if shape_rec.record[0]:  
                pts = shape_rec.shape.points
                prt = list(shape_rec.shape.parts) + [len(pts)]
                for i in range(len(prt) - 1):
                    for j in range(prt[i], prt[i+1]):
                        vertices.append(map(pts[j][0], pts[j][1]))
                    codes += [Path.MOVETO]
                    codes += [Path.LINETO] * (prt[i+1] - prt[i] -2)
                    codes += [Path.CLOSEPOLY]
                clip = Path(vertices, codes)
                clip = PathPatch(clip, transform = ax.transData)    
        for contour in cs.collections:
            contour.set_clip_path(clip)    
    # 内部函数
    def get_sql_data(self,start,end):      
        '''sql 获取数据'''
        tables = ['TAB_Aws2019','TAB_Mws2019']
        data_list = []
        sqltall = """select ta.IIiii,max(station.lat) as lat,max(station.lon) as lon,max(station.StationName) as StationName,
        max(station.City) as City,max(station.County) as County,max(station.Town) as Town,max(station.Village) as Village,max(station.Country) as Country, 
        max(fFy*1000+dFy) as fff ,min(VV) as view,max(T) as tmax,min(T) as tmin ,sum(RR) as rain,max(fFy) as wind
        from {table} as ta inner join TAB_StationInfo as station on ta.IIIII=station.IIiii  
        where (tTime between '{start}' and '{end}' and station.lon>119 and station.lon<122 and station.lat>27.5 and station.lat<29.5 ) 
        group by ta.IIiii"""
        for table in tables: 
            rsql = sqltall.format(start=start,end=end,table=table)
            data = pd.read_sql(rsql , con=self.conn)
            data_list.append(data)
        station_all = pd.concat(data_list)
        station_all['fff'] = station_all['fff'].values.astype(str)
        station_all['dFy'] = station_all['fff'].str.slice(-3)
        station_all["dFy"] = pd.to_numeric(station_all["dFy"])
        return station_all
    def extra_download(self,start,end,city,country):
        station_all = self.get_sql_data(start,end)
        station_all['lock'] = "true"
        if country=="all":
            country = city
            data = station_all[station_all['City']==city]
        else:
            country = country
            data = station_all[station_all['County']==country]
        output = data.to_json(orient='records',force_ascii=False)
        return output
    def get_plot_data(self,city,country,plot_data,plot_value):
        '''解析前段获取的数据'''
        x = []
        y = []
        z = []
        for i in range(len(plot_data)):
            if plot_data[i][plot_value]!=-9999.0:
                x.append(plot_data[i]['lon'])
                y.append(plot_data[i]['lat'])
                z.append(plot_data[i][plot_value]/10)
        lat = np.array(y)
        lon = np.array(x)
        Zi = np.array(z)
        data_max = max(Zi)
        data_min = min(Zi)
        np.set_printoptions(precision = 2)
        x = np.arange(120.0,122.0,0.05)
        y = np.arange(27.8,29.5,0.05)
        nx0 =len(x)
        ny0 =len(y)
        X, Y = np.meshgrid(x, y)#100*100
        P = np.array([X.flatten(), Y.flatten() ]).transpose()    
        Pi =  np.array([lon, lat ]).transpose()
        Z_linear = griddata(Pi, Zi, P, method = "nearest").reshape([ny0,nx0])
        data_xr = xr.DataArray(Z_linear, coords=[ y,x], dims=["lat", "lon"])
        return data_xr
    def plot(self,start,end,city,country,plot_data,plot_value,color_label):
        data_xr = self.get_plot_data(city,country,plot_data,plot_value)
        # 平滑
        #data_xr = scipy.ndimage.zoom(data_xr, 3)
        # ##########色标和大小#############################
        cmaps ,levels = self.colormap(plot_value,color_label)
        fig = plt.figure(figsize=[10,10]) 
        ax = fig.add_subplot(111)
        awash_da = self.city_shp(data_xr)
        lat = data_xr.lat
        lon = data_xr.lon
        m = Basemap(llcrnrlon=120.2,
            llcrnrlat=27.8,
            urcrnrlon=122,
            urcrnrlat=29.5,
            resolution = None, 
            projection = 'cyl')
        lons, lats = np.meshgrid(lon, lat)
        cs =m.contourf(lons,lats,data_xr,ax=ax, cmap=cmaps,levels =levels,add_labels=True)
        ##########标题#############################
        font = FontProperties(fname="simkai.ttf", size=14)
        label  = start + " 至 " + end + "   "  + "累积雨量"
        plt.text(120.2,29.4, label,fontsize=15, fontproperties=font)
        ##########标题#############################
        m.readshapefile(self.shp_path + 'taizhou','taizhou',color='k',linewidth=1.2)
        plt.axis('off')
        # 在图上绘制色标
        rect1 = [0.35, 0.25, 0.03, 0.12]         
        ax2 = plt.axes(rect1,frameon='False')
        ax2.set_xticks([])
        ax2.set_yticks([])
        ax2.spines['top'].set_visible(False)
        ax2.spines['bottom'].set_visible(False)
        ax2.spines['left'].set_visible(False)
        ax2.spines['right'].set_visible(False)
        m.colorbar(cs, location='right', size='30%', pad="-100%",ax = ax2)
        self.basemask(cs, ax, m, self.shp_path+'taizhou') 
        buffer = BytesIO()
        plt.savefig(buffer,bbox_inches='tight')  
        plot_img = buffer.getvalue()
        imb = base64.b64encode(plot_img) 
        ims = imb.decode()
        imd = "data:image/png;base64,"+ims
        return imd        
        
        
        
        
        

plot_data = [{"lat":28.6,"lon":121.5,"rain":-9999},{"lat":28.6,"lon":121.3,"rain":2000}]
start = '2019-08-08 09:00:00'  
end ='2019-08-09 09:00:00'
worker = station_plot()
city = "台州"
country = "all"
color_label = "rain_24hours"
# data = worker.extra_download(start,end,city,country)
plot_value = "rain"
data = worker.plot(start,end,city,country,plot_data,plot_value,color_label)